In [2]:
import json 

data = json.load(open('./output.json'))
json.dump(data, open('./output.json', 'w'), indent=4, ensure_ascii=False)

In [17]:
### 生成EE的数据集
import numpy as np
import json
data = json.load(open('./data/ee_convert/before.json'))
to_save=[]
for dialog in data:
    for turn in dialog['content']:
        keys = list(turn.keys())[:2]
        # t_ents = turn['info']['ents']
        sentence = ' '.join(turn[keys[0]]).split()
        item1={
            'sentence': sentence,
            'ner':[]
        }

        sentence = ' '.join(turn[keys[1]]).split()
        # for i, char in enumerate(sentence):
        #     if char == '|':
        #         sentence[i]=' '
        item2={
            'sentence':sentence,
            'ner':[]
        }

        # for ent in t_ents:
        #     ner = {
        #         'index':np.arange(ent['pos'][0][1], ent['pos'][0][2]).tolist(),
        #         'type':ent['type']
        #     }

        #     if ent['pos'][0][0] == 1:
        #         item1['ner'].append(ner)
        #     elif ent['pos'][0][0] == 2:
        #         item2['ner'].append(ner)
        
        # if item1['ner']:
        #     to_save.append(item1)
        # if item2['ner']:
        #     to_save.append(item2)

        to_save.append(item1)
        to_save.append(item2)

# np.random.seed(0)
# np.random.shuffle(to_save)
# np.random.shuffle(to_save)
# n = len(to_save)
# json.dump(to_save[:int(.8*n)], open('./data/seretod/train.json', 'w', encoding='utf8'), indent=4, ensure_ascii=False)
# json.dump(to_save[int(.8*n):int(.9*n)], open('./data/seretod/dev.json', 'w', encoding='utf8'), indent=4, ensure_ascii=False)
# json.dump(to_save[int(.9*n):], open('./data/seretod/test.json', 'w', encoding='utf8'), indent=4, ensure_ascii=False)
json.dump(to_save, open('./data/ee_convert/test_all.json', 'w', encoding='utf8'), indent=4, ensure_ascii=False)


In [1]:
### 用基线的测试方法测试此模型的输出结果
task = 'sf'
import json 
import re
labels = json.load(open(f'./data/{task}/test.json', encoding='utf8'))
bio_l = []
texts = []
for sent in labels:
    texts.append(''.join(sent['sentence']))
    res = ['O']*len(sent['sentence'])
    for ent in sent['ner']:
        for i in ent['index']:
            res[i] = f'I-{ent["type"]}'
        res[ent['index'][0]] = f'B-{ent["type"]}'

    bio_l.append(res)

output = json.load(open(f'./output/{task}/output.json', encoding='utf8'))
bio_o = []
for sent in output:
    res = ['O']*len(sent['sentence'])
    text = ''.join(sent['sentence'])
    for ent in sent['entity']:
        label = ''.join(ent['text'])
        span = re.search(label, text).span()
        for i in range(*span):
            res[i] = f"I-{ent['type'].upper()}"
        res[span[0]] = f"B-{ent['type'].upper()}"
    bio_o.append(res)


from seqeval.metrics import f1_score as span_f1_score
from seqeval.scheme import IOB2
micro_f1 = span_f1_score(bio_l, bio_o, mode='strict', scheme=IOB2) * 100.0
micro_f1


63.71959678165171

In [11]:
cases = []
for text, label, output in zip(texts, bio_l, bio_o):
    if label!=output:
        cases.append({
            'text': text,
            'label': ' '.join(label),
            'output': ' '.join(output)
        })
json.dump(cases, open('./case.json', 'w', encoding='utf8'), indent=4, ensure_ascii=False)


In [6]:
### 处理生成SF的数据集
import numpy as np
import json
data = json.load(open('./data/ee/data_label.json'))
to_save=[]
for dialog in data:
    for turn in dialog['content']:
        keys = list(turn.keys())[:2]
        t_triples = turn['info']['triples']
        
        item1={
            'sentence':' '.join(turn[keys[0]]).split(),
            'ner':[]
        }
        item2={
            'sentence':' '.join(turn[keys[1]]).split(),
            'ner':[]
        }

        for triple in t_triples:
            ner = {
                'index':np.arange(triple['pos'][1], triple['pos'][2]).tolist(),
                'type':triple['prop']
            }

            if triple['pos'][0] == 1:
                item1['ner'].append(ner)
            elif triple['pos'][0] == 2:
                item2['ner'].append(ner)
        
        if item1['ner']:
            to_save.append(item1)
        if item2['ner']:
            to_save.append(item2)

np.random.seed(0)
np.random.shuffle(to_save)
np.random.shuffle(to_save)
n = len(to_save)
json.dump(to_save[:int(.8*n)], open('./data/sf/train.json', 'w', encoding='utf8'), indent=4, ensure_ascii=False)
json.dump(to_save[int(.8*n):int(.9*n)], open('./data/sf/dev.json', 'w', encoding='utf8'), indent=4, ensure_ascii=False)
json.dump(to_save[int(.9*n):], open('./data/sf/test.json', 'w', encoding='utf8'), indent=4, ensure_ascii=False)
# json.dump(to_save, open('./data/all_data.json', 'w', encoding='utf8'), indent=4, ensure_ascii=False)



In [5]:
# 从W2NER到基线 EE

import json
before = json.load(open('/home/disk1/dgt2021/seretod/W2NER/data/ee_convert/before.json', encoding='utf8'))
infer_res = json.load(open('/home/disk1/dgt2021/seretod/W2NER/output/ee/converted_output.json', encoding='utf8'))
for i in range(1,20):
    infer_res.extend(json.load(open(f"./output/ee/converted_output{i}.json")))
for dialog in before:
    for turn in dialog['content']:
        ents = []
        keys = list(turn.keys())
        if turn[keys[0]] and infer_res:

            spk1 = infer_res.pop(0)
            sentence_text = ''.join(spk1['sentence'])
            assert sentence_text == turn[keys[0]].replace("\t","").replace(" ","").replace('\x1f','').replace("\u3000",''), (sentence_text, turn, 1)
            sentence_text = sentence_text.replace('\x7f','')
            turn[keys[0]] = sentence_text

            recorded = []
            for ent in spk1['entity']:
                ent_text = ''.join(ent['text'])
                if '\x7f' in ent_text:
                    continue
                if ent_text not in recorded:
                    begin = sentence_text.find(ent_text)
                    ents.append({
                        "pos":[[1, begin, begin+len(ent_text)]],
                        "type": ent['type'],
                        "name": ent_text
                    })
                    recorded.append(ent_text)

        if turn[keys[1]] and infer_res:

            spk2 = infer_res.pop(0)
            sentence_text = ''.join(spk2['sentence'])
            assert sentence_text == turn[keys[1]].replace("\t","").replace(" ","").replace('\x1f','').replace("\u3000",''), (sentence_text, turn, 2)
            sentence_text = sentence_text.replace('\x7f','')
            turn[keys[1]] = sentence_text

            recorded = []
            for ent in spk2['entity']:
                ent_text = ''.join(ent['text'])
                if '\x7f' in ent_text:
                    continue
                if ent_text not in recorded:
                    begin = sentence_text.find(ent_text)
                    ents.append({
                        "pos":[[2, begin, begin+len(ent_text)]],
                        "type": ent['type'],
                        "name": ent_text             
                    })
                    recorded.append(ent_text)
        turn["info"] = {}
        turn['info']['ents'] = ents
    if not infer_res:
        break
json.dump(before, open('./data/test_with_entity_mentions.json', 'w', encoding='utf8'), ensure_ascii=False, indent=4)
        


In [13]:
# 校正输出数据
import json
before = json.load(open('./data/test_with_entity_mentions.json', encoding='utf8'))
id = 0
for dialog in before:
    # for turn in dialog['content']:
    #     keys = list(turn.keys())
    #     for key in keys[:2]:

    #         turn[key] = turn[key].replace('\x7f', '')
    #     for i, ent in enumerate(turn['info']['ents']):
    #         if ent['name']=='\x7f':
    #             turn['info']['ents'].pop(i)
    dialog['id'] = f'testdata{id:0>8}'
    id+=1

json.dump(before, open('./data/test_with_entity_mentions.json', 'w', encoding='utf8'), ensure_ascii=False, indent=4)
json.dump(before, open('/home/disk1/dgt2021/seretod/baseline/Track1/baseline/data/test_with_entity_mentions.json', 'w', encoding='utf8'), ensure_ascii=False, indent=4)     


In [2]:
# 从W2NER到基线 SF

import json
before = json.load(open('/home/disk1/dgt2021/seretod/baseline/Track1/baseline/data/test_with_entity_coref.json', encoding='utf8'))
infer_res = []
for i in range(0,20):
    infer_res.extend(json.load(open(f"./output/sf/converted_output{i}.json")))
for dialog in before:
    for turn in dialog['content']:
        triples = []
        keys = list(turn.keys())
        if turn[keys[0]] and infer_res:

            spk1 = infer_res.pop(0)
            sentence_text = ''.join(spk1['sentence'])
            sentence_text = sentence_text.replace('\x7f','')
            assert sentence_text == turn[keys[0]].replace("\t","").replace(" ","").replace('\x1f','').replace("\u3000",''), (sentence_text, turn, 1)
            turn[keys[0]] = sentence_text

            recorded = []
            for ent in spk1['entity']:
                ent_text = ''.join(ent['text'])
                if '\x7f' in ent_text:
                    continue
                if ent_text not in recorded:
                    begin = sentence_text.find(ent_text)
                    triples.append({
                        "pos":[1, begin, begin+len(ent_text)],
                        "prop": ent['type'],
                        "value": ent_text
                    })
                    recorded.append(ent_text)

        if turn[keys[1]] and infer_res:

            spk2 = infer_res.pop(0)
            sentence_text = ''.join(spk2['sentence'])
            sentence_text = sentence_text.replace('\x7f','')
            assert sentence_text == turn[keys[1]].replace("\t","").replace(" ","").replace('\x1f','').replace("\u3000",''), (sentence_text, turn, 2)
            turn[keys[1]] = sentence_text

            recorded = []
            for ent in spk2['entity']:
                ent_text = ''.join(ent['text'])
                if '\x7f' in ent_text:
                    continue
                if ent_text not in recorded:
                    begin = sentence_text.find(ent_text)
                    triples.append({
                        "pos":[2, begin, begin+len(ent_text)],
                        "prop": ent['type'],
                        "value": ent_text             
                    })
                    recorded.append(ent_text)
        turn['info']['triples'] = triples
    if not infer_res:
        break
json.dump(before, open('./data/test_with_triples.json', 'w', encoding='utf8'), ensure_ascii=False, indent=4)
json.dump(before, open('/home/disk1/dgt2021/seretod/baseline/Track1/baseline/data/test_with_triples.json', 'w', encoding='utf8'), ensure_ascii=False, indent=4)

In [21]:
#把测试集分成小份
import json
import numpy as np
data = json.load(open("./data/ee_convert/test_all.json", encoding="utf8"))
n = len(data)
split = np.linspace(0,n,21,dtype=int).tolist()
for i in range(20):
    json.dump(data[split[i]:split[i+1]], open(f"./data/ee_convert/test{i}.json", "w", encoding="utf8"), indent=4, ensure_ascii=False)

In [12]:
i=123
print(f'abc{i:0>8}')

abc00000123
